In [1]:
# train_model.py
import pandas as pd
import numpy as np
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
import joblib

# 1. Veriyi Yükle
df = pd.read_csv("twitter.csv")

# 2. Veri Temizleme Fonksiyonu
def clean_text(text):
    text = str(text).lower() # Küçük harfe çevir
    text = re.sub('\[.*?\]', '', text) # Köşeli parantez içindekileri sil
    text = re.sub('https?://\S+|www\.\S+', '', text) # URL'leri sil
    text = re.sub('<.*?>+', '', text) # HTML etiketlerini sil
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text) # Noktalama işaretlerini sil
    text = re.sub('\n', '', text) # Yeni satır karakterlerini sil
    text = re.sub('\w*\d\w*', '', text) # Rakam içeren kelimeleri sil
    return text

# Temizleme işlemini uygula
print("Veriler temizleniyor...")
df['clean_tweet'] = df['tweet'].apply(clean_text)

# 3. Model Hazırlığı
X = df['clean_tweet']
y = df['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. Pipeline Kurulumu (Vektörleştirme + Model)
# CountVectorizer metni sayıya çevirir, DecisionTree sınıflandırır.
pipeline = Pipeline([
    ('vectorizer', CountVectorizer(stop_words='english')), # İngilizce etkisiz kelimeleri (the, a, in) çıkar
    ('classifier', DecisionTreeClassifier(random_state=42))
])

# 5. Modeli Eğitme
print("Model eğitiliyor...")
pipeline.fit(X_train, y_train)

# 6. Değerlendirme
y_pred = pipeline.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(classification_report(y_test, y_pred))

# 7. Modeli Kaydetme
joblib.dump(pipeline, 'hate_speech_model.pkl')
print("Model 'hate_speech_model.pkl' olarak kaydedildi.")

Veriler temizleniyor...
Model eğitiliyor...
Accuracy: 0.8735
              precision    recall  f1-score   support

           0       0.37      0.32      0.34       290
           1       0.93      0.92      0.92      3832
           2       0.79      0.84      0.81       835

    accuracy                           0.87      4957
   macro avg       0.70      0.69      0.69      4957
weighted avg       0.87      0.87      0.87      4957

Model 'hate_speech_model.pkl' olarak kaydedildi.
